In [1]:
#!/usr/bin/env python
# coding: utf-8

# Import required libraries and modules
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from astropy.coordinates import SkyCoord
import astropy.units as u
from tqdm import tqdm

Generating truth sample for the injected DC2 run `u/elhoward/DM-45116/w_2024_33/DC2-with-injection`

In [10]:

# Define a threshold for matching errors
space_match_threshold = 1 * u.arcsec
MJD_tolerance = 31./(24*3600) #31 sec in units of day 

# Define file paths.
sum_path = {}
sum_path["star"] = '../truth_star/truth_star_summary_v1-0-0.parquet'
#'/sdf/data/rubin/shared/dc2_run2.2i_truth/truth_star/truth_star_summary_v1-0-0.parquet'
sum_path["sn"] = "../truth_sn/truth_sn_summary_v1-0-0.parquet" 
#'/sdf/data/rubin/shared/dc2_run2.2i_truth/truth_sn/truth_sn_summary_v1-0-0.parquet'

# injected sources
sum_path["inject"] = '../test-DC2-injection-catalog.csv.gz'

var_path = {}
var_path["star"] = '../truth_star/truth_star_variability_v1-0-0.parquet'
#'/sdf/data/rubin/shared/dc2_run2.2i_truth/truth_star/truth_star_variability_v1-0-0.parquet'
var_path["sn"] = '../truth_sn/truth_sn_variability_v1-0-0.parquet'
#'/sdf/data/rubin/shared/dc2_run2.2i_truth/truth_sn/truth_sn_variability_v1-0-0.parquet'



detection_csv_pth = "../all_diasources.csv.gz" #'exported_sources.csv'

#get DIA detections
dia_detections = pd.read_csv(detection_csv_pth, index_col="diaSourceId") #formerly known as exported_csv


dia_detections

ra        dec  midpointMjdTai
diaSourceId                                             
1253071204122625    59.253969 -35.865458    59583.120963
1253071204122626    59.352831 -35.824191    59583.120963
1253071204122627    59.322393 -35.837479    59583.120963
1253071204122628    59.187448 -35.894133    59583.120963
1253071204122629    59.357712 -35.822867    59583.120963
...                       ...        ...             ...
266819038615699894  58.511378 -36.092158    60309.228989
266819038615699895  58.506497 -36.097308    60309.228989
266819038615699897  58.470790 -36.139579    60309.228989
266819038615699898  58.565491 -36.033826    60309.228989
266819038615699899  58.525060 -36.079423    60309.228989

[3508059 rows x 3 columns]

## load truth catalogs

In [11]:
# Get mind and max ra and dec values to filter out unnecessary records.
max_exp_ra, min_exp_ra = dia_detections.ra.max(), dia_detections.ra.min()
max_exp_dec, min_exp_dec = dia_detections.dec.max(), dia_detections.dec.min()


catalog = {}
result_sum = {}

# Stage 1: Match sources in Space.
for s in ["star", "sn", "inject"]:
    # Read Parquet and CSV files to begin ground truth derivation.
    if sum_path[s].endswith('parquet'):
        result_sum[s] = pd.read_parquet(sum_path[s])
    else:
        result_sum[s] = pd.read_csv(sum_path[s])

    # Keep only those records from summary tables which are within the max ra and dec values in the exported sources.
    result_sum[s] = result_sum[s][(result_sum[s]['ra'] >= min_exp_ra) & (result_sum[s]['ra'] <= max_exp_ra) &\
                                    (result_sum[s]['dec'] >= min_exp_dec) & (result_sum[s]['dec'] <= max_exp_dec)]


    # Initialize astropy.coordinates.SkyCoord class for matching in space.
    catalog[s] = SkyCoord(ra=result_sum[s].ra, dec=result_sum[s].dec, unit=u.deg)

# Match exported sources with stars and supernovae.
detections_cat = SkyCoord(ra=dia_detections.ra, dec=dia_detections.dec, unit=u.deg)

In [15]:
result_sum["inject"]

injection_id         ra        dec        mag source_type
0                 0  58.758793 -35.758146  19.036632        Star
1                 1  57.274012 -37.012349  22.536632        Star
2                 2  56.182822 -36.767262  20.786632        Star
3                 3  57.626783 -36.773840  24.286632        Star
4                 4  57.134388 -36.457476  18.161632        Star
...             ...        ...        ...        ...         ...
47689         47689  57.480201 -35.786387  23.023158        Star
47690         47690  58.508305 -35.913646  21.273158        Star
47691         47691  57.337736 -36.804169  24.773158        Star
47692         47692  59.198191 -35.984369  18.648158        Star
47693         47693  56.516517 -36.450343  22.148158        Star

[46907 rows x 5 columns]

In [13]:
# By default, set on_source = 0 and real=0 (bogus) for all values in the exported sources.
dia_detections['on_source'] = 0
dia_detections['real'] = np.nan
dia_detections['type'] = None

## spatial crossmatch

In [17]:
# we want to put the DIA catalog first so we get a yes or no truth match for every DIASource
star_idx, star_d2d, star_d3d = detections_cat.match_to_catalog_sky(catalog['star'])
sn_idx, sn_d2d, sn_d3d = detections_cat.match_to_catalog_sky(catalog['sn'])
inj_idx, inj_d2d, inj_d3d = detections_cat.match_to_catalog_sky(catalog['inject'])


star_mask = star_d2d < space_match_threshold #remove matches that are too far
sn_mask = sn_d2d < space_match_threshold #remove matches that are too far
inj_mask = inj_d2d < space_match_threshold #remove matches that are too far


print(f"{np.sum(star_mask)} of {len(detections_cat)} matched stars after applying spatial threshold")
print(f"{np.sum(sn_mask)} of {len(detections_cat)} matched sne after applying spatial threshold")
print(f"{np.sum(inject_mask)} of {len(detections_cat)} matched injections after applying spatial threshold")

211188 of 3508059 matched stars after applying spatial threshold
2829 of 3508059 matched sne after applying spatial threshold
3215924 of 3508059 matched injections after applying spatial threshold


In [31]:


# Get all matched stars
matched_star_idx = star_idx[star_mask] #index in stars of matched dia_detections
print(f"Number of matched stars in Stage #1: {len(matched_star_idx)}")

# Get all matched supernovae
matched_sn_idx = sn_idx[sn_mask] #index in sn_cat of matched dia_detections
print(f"Number of matched sne in Stage #1: {len(matched_sn_idx)}")

# Get all matched injections
matched_inj_idx = inj_idx[inj_mask] #index in sn_cat of matched dia_detections
print(f"Number of matched injections in Stage #1: {len(matched_inj_idx)}")

dia_idx_stars = dia_detections.index[star_mask]
dia_idx_sn = dia_detections.index[sn_mask]
dia_idx_inj = dia_detections.index[inj_mask]


# Assign the variability sources catalog id to the detections
dia_detections["id"] = None
dia_detections.loc[dia_idx_stars,"id"] = result_sum["star"].iloc[matched_star_idx]["id"].to_numpy()
dia_detections.loc[dia_idx_sn,"id"] = result_sum['sn'].iloc[matched_sn_idx]["id"].to_numpy()
dia_detections.loc[dia_idx_inj,"id"] = result_sum['inject'].iloc[matched_inj_idx]["injection_id"].to_numpy()



Number of matched stars in Stage #1: 211188
Number of matched sne in Stage #1: 2829
Number of matched injections in Stage #1: 3215924


In [32]:
dia_detections.head()

ra        dec  midpointMjdTai           id
diaSourceId                                                        
1253071204122625  59.253969 -35.865458    59583.120963        17968
1253071204122626  59.352831 -35.824191    59583.120963        17550
1253071204122627  59.322393 -35.837479    59583.120963        39378
1253071204122628  59.187448 -35.894133    59583.120963  31107842865
1253071204122629  59.357712 -35.822867    59583.120963        12941

In [ ]:
# for now, classify anything that doesn't match truth summary or injection tables as bogus.  
# Call anything injected real.  
# Ignore the truth summary matches.

In [33]:
dia_detections['real'] = np.nan

In [34]:
dia_detections.loc[~(star_mask | sn_mask | inj_mask), 'real'] = 0
dia_detections.loc[dia_idx_inj, 'real'] = 1

In [35]:
np.sum(dia_detections['real'].isna())

204037

In [36]:
np.sum(dia_detections['real'] == 0)

88098

In [37]:
np.sum(dia_detections['real'] == 1)

3215924

In [38]:
dia_detections.head()

ra        dec  midpointMjdTai           id  real
diaSourceId                                                              
1253071204122625  59.253969 -35.865458    59583.120963        17968   1.0
1253071204122626  59.352831 -35.824191    59583.120963        17550   1.0
1253071204122627  59.322393 -35.837479    59583.120963        39378   1.0
1253071204122628  59.187448 -35.894133    59583.120963  31107842865   NaN
1253071204122629  59.357712 -35.822867    59583.120963        12941   1.0

duplicates code from [here](https://github.com/lsst/analysis_ap/blob/f648f42b89952fea3f8aa3e5145f8f1c87867698/python/lsst/analysis/ap/plotImageSubtractionCutouts.py#L680C1-L682C37)


In [41]:
def chunker(id, size):
    return (id // size)*size

def cutout_path(id, size=5000):
    return f'numpy/{chunker(id, size)}/{id}_sci_51.npy'

In [46]:
cutout_paths = pd.Series([cutout_path(i) for i in dia_detections.index.to_numpy()], index=dia_detections.index,
                         name='cutout_path')

In [49]:
dia_detections.join(cutout_paths).to_csv('../diasources_with_labels.csv.gz')

** code below is unused for now **

In [ ]:
# By default, set on_source = 0 for all values in the exported sources.
dia_detections['on_source'] = 0
dia_detections['type'] = None

In [8]:
# The spatially matched detections get on_source = 1

dia_detections.loc[dia_idx_sn, "on_source"] = 1
dia_detections.loc[dia_idx_stars , "on_source"] = 1
dia_detections.loc[dia_idx_sn, "type"] = "sn"
dia_detections.loc[dia_idx_stars, "type"] = "star"
dia_detections

ra        dec  midpointMjdTai  type  on_source  \
diaSourceId                                                                 
1257927201521665    55.760339 -32.260622    59583.125051  star          1   
1257927201521666    55.674078 -32.283857    59583.125051  None          0   
1257927201521667    55.552914 -32.306395    59583.125051  None          0   
1257927201521668    55.547689 -32.309278    59583.125051  None          0   
1257927201521669    55.570127 -32.306400    59583.125051  None          0   
...                       ...        ...             ...   ...        ...   
660667525163384915  55.889519 -32.485637    61392.194195  star          1   
661047079476396040  55.863218 -32.167598    61393.204087  star          1   
662500331589992590  55.971559 -32.358853    61404.195949  star          1   
662500331589992596  55.881022 -32.482719    61404.195949  star          1   
662500331589992604  55.875809 -32.505451    61404.195949  star          1   

                    real           id  
diaSourceId                            
1257927201521665       0  30321355720  
1257927201521666       0         None  
1257927201521667       0         None  
1257927201521668       0         None  
1257927201521669       0         None  
...                  ...          ...  
660667525163384915     0  31411443281  
661047079476396040     0  31102009372  
662500331589992590     0  31405685742  
662500331589992596     0  31411442918  
662500331589992604     0  31107747991  

[25446 rows x 7 columns]

In [9]:
# Print a summary at the end of first round of matching.
print("Summary at the end of First Stage:")
print(f"detections on a source", dia_detections["on_source"].sum(), "\n")
print(f"class detection: {dia_detections.groupby('type').count().iloc[:,0]}")

Summary at the end of First Stage:
detections on a source 7301 

class detection: type
sn       146
star    7155
Name: ra, dtype: int64


## Stage 2: Match sources in time.

We are deferring the DC2 truth match for now because the number of MJD matches is not large.

In [11]:

matched = {}
matched["sn"] = dia_detections.loc[dia_idx_sn]
matched["star"] = dia_detections.loc[dia_idx_stars]

# make a column to report cases where there is no variability entry for that truth id
dia_detections['not_in_truth_var'] = False
# make a column to store minimum MJD difference
dia_detections['min_mjd_offset'] = np.inf

for s in ["sn"]:
#for s in ["sn", "star"]:


    print(f"working on class: {s}")
    
    # Get a list of all the unique MJDs of sources that matched in the previous stage for the object type.
    mjd_matched_in_space = matched[s].midpointMjdTai.unique()

    # Get min and max MJD values required for matching.
    max_mjd, min_mjd = mjd_matched_in_space.max(), mjd_matched_in_space.min()

    # Read star/sn lightcurve variability parquet for the object type
    df_var = pd.read_parquet(var_path[s])
    
    # Filter out records with unwanted MJDs.
    df_var = df_var[(df_var.MJD >= min_mjd) & (df_var.MJD <= max_mjd)]
    print(f"need to examine {len(df_var)} variability entries")
    
    for detected in tqdm(matched[s].index): #loop over indices of on_source detection 
        mask_matching_ids = df_var.id == matched[s].loc[detected, 'id'] #mask for sources with matching id in variability file 
        if np.sum(mask_matching_ids) == 0:
            dia_detections.loc[detected, 'not_in_truth_var'] = True
        else:
            dia_detections.loc[detected, 'min_mjd_offset'] = np.min(np.abs(df_var[mask_matching_ids].MJD - 
                                                                           matched[s].loc[detected].midpointMjdTai))
            dia_detections.loc[detected, "real"] = dia_detections.loc[detected, 'min_mjd_offset'] <= MJD_tolerance
    del df_var

dia_detections[dia_detections.real == 1]

working on class: sn
need to examine 12757691 variability entries


100%|█████████████████████████████████████████| 146/146 [00:01<00:00, 91.96it/s]


ra        dec  midpointMjdTai type  on_source  \
diaSourceId                                                                
85627704270914249   55.734059 -32.316071    59791.360480   sn          1   
95252785037049908   55.734090 -32.316053    59813.364076   sn          1   
97640275215646752   55.734070 -32.316099    59821.276470   sn          1   
97657362206163020   55.734072 -32.316068    59821.291001   sn          1   
161196052894974023  55.685292 -32.308088    60001.075928   sn          1   
164379687202586649  55.685270 -32.308128    60009.044807   sn          1   
278114224394207329  55.746318 -32.272641    60338.139110   sn          1   
286057742248968251  55.746346 -32.272636    60362.072829   sn          1   
341338831640854570  55.756720 -32.300051    60528.318074   sn          1   
341357091157442660  55.756743 -32.300082    60528.333570   sn          1   
342457652904722669  55.756765 -32.300058    60530.355550   sn          1   
342962846285430848  55.756760 -32.300047    60531.327979   sn          1   
343039123830865946  55.756746 -32.300077    60531.394166   sn          1   
345230015102713919  55.756748 -32.300045    60536.295524   sn          1   
348166212049436764  55.756753 -32.300088    60542.294638   sn          1   
348186613144092681  55.756739 -32.300077    60542.312628   sn          1   
348223636835926049  55.756781 -32.300101    60542.345045   sn          1   
351362700657295472  55.756703 -32.300073    60549.272587   sn          1   
351363319669456940  55.756695 -32.300074    60549.273034   sn          1   
351363794800214114  55.756753 -32.300068    60549.273482   sn          1   
351379903611928620  55.756702 -32.300083    60549.287105   sn          1   
355221811450872090  55.756784 -32.300028    60559.302831   sn          1   
405771409713791040  55.830173 -32.331817    60696.136692   sn          1   
487997976627642371  55.956356 -32.368539    60904.380613   sn          1   
491618669491650998  55.956362 -32.368529    60912.324638   sn          1   
496114968211488916  55.753912 -32.253740    60937.263806   sn          1   
636277479094353933  55.945609 -32.353280    61318.333201   sn          1   
636297807174565911  55.945607 -32.353279    61318.351789   sn          1   
636760137318531161  55.945618 -32.353251    61319.363939   sn          1   
645816020564443201  55.945600 -32.353267    61343.105723   sn          1   
660667525163384854  55.945622 -32.353261    61392.194195   sn          1   
93738742493216770   55.734081 -32.316066    59810.381776   sn          1   
225905618777538634  55.619536 -32.227419    60179.353690   sn          1   
278114224394207312  55.619952 -32.235140    60338.139110   sn          1   
289069573032902730  55.746311 -32.272632    60369.030714   sn          1   
343017058973253667  55.929655 -32.427467    60531.374974   sn          1   
343018160632365243  55.756722 -32.300056    60531.375875   sn          1   
407778182180110377  55.830114 -32.331786    60701.123060   sn          1   
479854721318780936  55.823223 -32.241916    60887.403224   sn          1   
487999617842020420  55.956362 -32.368533    60904.381976   sn          1   
496099340436111427  55.753915 -32.253749    60937.250689   sn          1   
514655207620083750  55.927975 -32.498102    60991.073999   sn          1   
638865685442723935  55.945603 -32.353276    61324.220001   sn          1   

                    real             id  not_in_truth_var  min_mjd_offset  
diaSourceId                                                                
85627704270914249   True  5056389737494             False        0.000347  
95252785037049908   True  5056389737494             False        0.000347  
97640275215646752   True  5056389737494             False        0.000347  
97657362206163020   True  5056389737494             False        0.000347  
161196052894974023  True  4224016890902             False        0.000347  
164379687202586649  True  4224016890902             False        0.000347 

In [14]:
dia_detections.loc[(dia_detections.real == 0) & (dia_detections.on_source == 1) & (dia_detections.type == 'sn'), 
['type','not_in_truth_var', 'min_mjd_offset'] ].sort_values('min_mjd_offset')

type  not_in_truth_var  min_mjd_offset
diaSourceId                                              
85620264850686737    sn             False        0.006872
662500331589992579   sn             False       10.015978
514651499452694538   sn             False       14.951599
514655207620083813   sn             False       14.954745
514672918991470638   sn             False       14.969620
...                 ...               ...             ...
372269053118513218   sn              True             inf
242418712005574688   sn              True             inf
242392416068304901   sn              True             inf
642315619533848634   sn              True             inf
645816020564443154   sn              True             inf

[103 rows x 3 columns]

next step: look at image stamps to see if we can understand the behavior of some of this cases where there's no match in variability (either by MJD or by id).

At a first look at the SN, these look like boguses apart from 85620264850686737 which is only an hour or so from a truth MJD.  Lots of bad galaxy supernova.

Need to check missing detections...  we don't have cutouts for those

add a bunch of fakes to the DC2 images to enhance truth sample in the short term, to get us a functional model for comcam

In [ ]:
variable stars: feed in the ML features from the dipole fitter?

figure out by MJD how many SN we have have epochs we didn't process